In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load your fine-tuned model and tokenizer
model_path = "testmodel"  # Change this to your actual model path

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

In [8]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")

Device set to use cuda:0


In [9]:
text = """Threat intelligence company GreyNoise warns that a critical PHP remote code execution vulnerability that impacts Windows systems is now under mass exploitation.

Tracked as CVE-2024-4577, this PHP-CGI argument injection flaw was patched in June 2024 and affects Windows PHP installations with PHP running in CGI mode. Successful exploitation enables unauthenticated attackers to execute arbitrary code and leads to complete system compromise following successful exploitation.

A day after PHP maintainers released CVE-2024-4577 patches on June 7, 2024, WatchTowr Labs released proof-of-concept (PoC) exploit code, and the Shadowserver Foundation reported observing exploitation attempts.

GreyNoise's warning comes after Cisco Talos revealed earlier that an unknown attacker had exploited the same PHP vulnerability to target Japanese organizations since at least early January 2025.

While Talos observed the attackers attempting to steal credentials, it believes their goals extend beyond just credential harvesting, based on post-exploitation activities, which include establishing persistence, elevating privileges to SYSTEM level, deployment of adversarial tools and frameworks, and usage of "TaoWu" Cobalt Strike kit plugins."""
#text = "The redacted court order reveals that the threat actors breached Genea's network on January 31, 2025, through a Citrix server. Subsequently, they gained access to the company's primary file server, domain controller, backup program, and BabySentry primary patient management system. Two weeks later, on February 14, the attackers exfiltrated 940.7GB of data from Genea's compromised systems to a DigitalOcean cloud server under their control."
# Run the model for Named Entity Recognition
predictions = ner_pipeline(text)

# Print results
for entity in predictions:
    print(entity)

{'entity_group': 'ORG', 'score': np.float32(0.21117336), 'word': 'GreyNoise', 'start': 28, 'end': 37}
{'entity_group': 'ORG', 'score': np.float32(0.18722235), 'word': 'CVE', 'start': 173, 'end': 176}
{'entity_group': 'CVE', 'score': np.float32(0.4279956), 'word': '- 2024 - 4577', 'start': 176, 'end': 186}
{'entity_group': 'ORG', 'score': np.float32(0.2256279), 'word': 'PHP', 'start': 193, 'end': 196}
{'entity_group': 'CVE', 'score': np.float32(0.1954003), 'word': '-', 'start': 196, 'end': 197}
{'entity_group': 'ORG', 'score': np.float32(0.17721191), 'word': 'CGI', 'start': 197, 'end': 200}
{'entity_group': 'ORG', 'score': np.float32(0.17379494), 'word': 'CGI', 'start': 308, 'end': 311}
{'entity_group': 'ORG', 'score': np.float32(0.22178522), 'word': 'PHP', 'start': 490, 'end': 493}
{'entity_group': 'ORG', 'score': np.float32(0.17325987), 'word': 'CVE', 'start': 515, 'end': 518}
{'entity_group': 'CVE', 'score': np.float32(0.42585748), 'word': '- 2024 - 4577', 'start': 518, 'end': 528}
{